### 1. Importing required libraries and loading data

In [1]:
import pandas as pd
import numpy as np

In [2]:
file = './customer_order_data_input.csv'

data = pd.read_csv(file, parse_dates=['OrderDate'])

In [3]:
data

,OrderId,CustomerId,OrderAmount,OrderDate
0,Order-0000001,Cust-00001,406,2020-06-18
1,Order-0000002,Cust-00004,254,2020-07-02
2,Order-0000003,Cust-00002,282,2020-07-04
3,Order-0000004,Cust-00005,12268,2020-07-04
4,Order-0000005,Cust-00008,169,2020-07-05
...,...,...,...,...
788557,Order-0788558,Cust-07465,2284,2023-06-14
788558,Order-0788559,Cust-09602,4812,2023-06-14
788559,Order-0788560,Cust-09602,252,2023-06-14
788560,Order-0788561,Cust-27623,134,2023-06-14


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788562 entries, 0 to 788561
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   OrderId      788562 non-null  object        
 1   CustomerId   788562 non-null  object        
 2   OrderAmount  788562 non-null  int64         
 3   OrderDate    788562 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 24.1+ MB


### 2. Calculating RFM values (R = "recency", F = "frequency" and M = "Monetary Value")

In [5]:
recency = data.groupby(by='CustomerId', as_index=False)['OrderDate'].max()
recency.columns = ['CustomerId', 'LastOrderDate']
recent_date = recency['LastOrderDate'].max()
recency['Recency'] = recency['LastOrderDate'].apply(lambda x: (recent_date - x).days)
recency

,CustomerId,LastOrderDate,Recency
0,Cust-00001,2022-12-24,172
1,Cust-00002,2023-03-09,97
2,Cust-00003,2023-05-23,22
3,Cust-00004,2023-01-02,163
4,Cust-00005,2023-06-06,8
...,...,...,...
49995,Cust-49996,2023-02-09,125
49996,Cust-49997,2023-03-28,78
49997,Cust-49998,2022-01-05,525
49998,Cust-49999,2023-04-21,54


In [6]:
frequency = data.drop_duplicates().groupby(by='CustomerId', as_index=False)['OrderDate'].count()
frequency.columns = ['CustomerId', 'Frequency']
frequency

,CustomerId,Frequency
0,Cust-00001,8
1,Cust-00002,12
2,Cust-00003,240
3,Cust-00004,12
4,Cust-00005,56
...,...,...
49995,Cust-49996,2
49996,Cust-49997,8
49997,Cust-49998,1
49998,Cust-49999,7


In [7]:
monetary = data.groupby(by='CustomerId',
                        as_index=False)['OrderAmount'].sum()
monetary.columns = ['CustomerId', 'Monetary']
#monetary['Monetary'] = round(monetary['Monetary'],1)
monetary

,CustomerId,Monetary
0,Cust-00001,2965
1,Cust-00002,45470
2,Cust-00003,147312
3,Cust-00004,14433
4,Cust-00005,131280
...,...,...
49995,Cust-49996,8913
49996,Cust-49997,23827
49997,Cust-49998,756
49998,Cust-49999,3108


### 3. Merging all RFM values and calculating RFM scores (ranks)*
*Mote: in here, the range of scores go from (almost) 0 to 100. This range might change depending on context (for example, from 1 to 5, or from 1 to 10).

In [8]:
rfm = recency.merge(frequency, on='CustomerId')
rfm = rfm.merge(monetary, on='CustomerId')
rfm.drop(columns=['LastOrderDate'], axis=1, inplace=True)
rfm

,CustomerId,Recency,Frequency,Monetary
0,Cust-00001,172,8,2965
1,Cust-00002,97,12,45470
2,Cust-00003,22,240,147312
3,Cust-00004,163,12,14433
4,Cust-00005,8,56,131280
...,...,...,...,...
49995,Cust-49996,125,2,8913
49996,Cust-49997,78,8,23827
49997,Cust-49998,525,1,756
49998,Cust-49999,54,7,3108


In [9]:
rfm['R_score'] = rfm['Recency'].rank(ascending=False)
rfm['F_score'] = rfm['Frequency'].rank(ascending=True)
rfm['M_score'] = rfm['Monetary'].rank(ascending=True)

rfm['R_score'] = round((rfm['R_score']/rfm['R_score'].max())*100, 5)
rfm['F_score'] = round((rfm['F_score']/rfm['F_score'].max())*100, 5)
rfm['M_score'] = round((rfm['M_score']/rfm['M_score'].max())*100, 5)

rfm

,CustomerId,Recency,Frequency,Monetary,R_score,F_score,M_score
0,Cust-00001,172,8,2965,39.28110,68.422,16.376
1,Cust-00002,97,12,45470,54.57566,79.162,89.999
2,Cust-00003,22,240,147312,84.23512,99.273,97.776
3,Cust-00004,163,12,14433,41.20843,79.162,60.142
4,Cust-00005,8,56,131280,95.65800,96.329,97.440
...,...,...,...,...,...,...,...
49995,Cust-49996,125,2,8913,47.88404,16.241,44.104
49996,Cust-49997,78,8,23827,61.06714,68.422,76.368
49997,Cust-49998,525,1,756,10.70839,3.935,3.984
49998,Cust-49999,54,7,3108,68.16704,63.962,17.196


In [10]:
rfm.describe()

,Recency,Frequency,Monetary,R_score,F_score,M_score
count,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.000000,50000.000000
mean,180.871540,15.771240,2.640756e+04,50.035524,50.001000,50.001000
std,182.649831,123.971349,1.228850e+05,28.887435,28.735875,28.867802
min,0.000000,1.000000,1.000000e+02,0.002000,3.935000,0.002000
25%,38.000000,3.000000,4.516000e+03,24.951220,30.522000,25.007000
50%,116.000000,5.000000,1.069550e+04,50.135590,51.272000,50.000000
75%,258.000000,10.000000,2.269525e+04,75.206890,74.910000,74.999750
max,687.000000,20425.000000,1.473686e+07,100.000000,100.000000,100.000000


### 4. Saving to csv file to be feed into clustering model

In [11]:
rfm.to_csv('./customer_rfm_scores.csv')